# Importations

In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPool2D, Flatten
from keras import optimizers
from keras import losses
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random
import pandas as pd
import matplotlib.image as mpimg
from os import getcwd, chdir, mkdir
import scipy as sp
import matplotlib.pyplot as plt
import cv2
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 
from matplotlib.pyplot import imread 
from pylab import ginput
from scipy import signal
import matplotlib
from scipy.ndimage import convolve 

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

# Lecture des données

In [ ]:
dataframe1 = pd.read_csv("D:/mias-mammography/Info.txt")
dataframe1.to_csv("./Info.csv", 
                  index = None)

In [ ]:
data_spec=pd.read_csv("../input/mias-mammography/Info.txt",sep=" ")
data_spec=data_spec.drop('Unnamed: 7',axis=1)
data_spec

# Suppression des mammographies dites "Saines"

In [ ]:
data_spec.dropna(subset = ["SEVERITY"], inplace=True) 
data_spec.reset_index(inplace = True)
data_spec

In [ ]:
data_spec=data_spec.drop([3], axis=0)
data_spec.reset_index(inplace = True)
data_spec

In [ ]:
dspec = {}
for i in range(len(data_spec)):
    dspec[i] = data_spec.REFNUM[i]
dspec

# Correspondance Bégigne, Maligne vers 1,0

In [ ]:
lb = []
for i in range(len(data_spec)):
    if data_spec.SEVERITY[i] == 'B':
        lb.append(1)
    else:
        lb.append(0)
lb=np.array(lb)
img_name = []
path = 'D:/mias-mammography/all-mias/'
for i in range(len(lb)):
        img_name.append(path + data_spec.REFNUM[i]+ '.pgm')
img_name = np.array(img_name)

In [ ]:
def view_25_random_image():
    fig = plt.figure(figsize = (15, 10))
    for i in range(25):
        rand = random.randint(0,len(lb))
        ax = plt.subplot(5, 5, i+1)
    
        img = cv2.imread(img_name[rand], 0)
        img = cv2.resize(img, (256,256))
        if lb[rand] == 1:
            plt.title('B')
        else:
            plt.title('M')
        plt.tight_layout()
        plt.axis('off')
        plt.imshow(img)
    fig.savefig('random_25_image_fig.png')

random_images = view_25_random_image()

# Lecture et Zoom des images

In [ ]:
img_path = []
last_lb = []
for i in range(len(img_name)):
    
    img = cv2.imread(img_name[i], 0)
    img = cv2.resize(img, (224,224))
    rows, cols= img.shape
    for angle in range(360):
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)    #Rotate 0 degree
            img_rotated = cv2.warpAffine(img, M, (224, 224))
            img_path.append(img_rotated)
            if lb[i] == 1:
                last_lb.append(1)
            else:
                last_lb.append(0)
last_lb = np.array(last_lb)
img_path = np.array(img_path)

# Séparation Train/Test du dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(img_path, last_lb, test_size = 0.2, random_state = 42)

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
(a,b,c)=x_train.shape # (35136, 224, 224)
x_train = np.reshape(x_train, (a, b, c, 1)) # 1 for gray scale
(a, b, c)=x_test.shape
x_test = np.reshape(x_test, (a, b, c, 1))

# Augmentation de la quantité de données (Data Augmentation)

In [ ]:
from TensorFlow keras preprocessing image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [ ]:
datagen.fit(x_train)

# Définition et création du modèle

In [ ]:
def model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), padding = 'same',activation='relu',input_shape=(224, 224, 1)))
    model.add(Conv2D(64, kernel_size=(3,3), padding = 'same',activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(3,3), padding = 'same',activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
  
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
model = model()
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=0,restore_best_weights=True, verbose=1)

check_point_filepath = './'

model_check_point = ModelCheckpoint(filepath =check_point_filepath, monitor='val_loss', verbose=1, save_best_only=True,
                                    save_weights_only=False, mode='auto', save_freq='epoch')

In [ ]:
hist = model.fit(datagen.flow(x_train, y_train, batch_size=32,
                 subset='training'),
                 steps_per_epoch = len(x_train)/32,
                 validation_split=0.3,
                 epochs=5,
                 batch_size=8,
                 callbacks=[early_stop, model_check_point],validation_data=(x_test,y_test))

In [ ]:
from keras.models import model_from_yaml
def saveModel(model, savename):
  # serialize model to YAML
  model_yaml = model.to_yaml()
  with open(savename+".yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
    print("Yaml Model ",savename,".yaml saved to disk")
  # serialize weights to HDF5
  model.save_weights(savename+".h5")
  print("Weights ",savename,".h5 saved to disk")
saveModel(model, save1)

# Fonctions de visualisation de la précision et de la perte du modèle

In [ ]:
loss_value , accuracy = model.evaluate(x_test, y_test)

print('Test_loss_value = ' +str(loss_value))
print('test_accuracy = ' + str(accuracy))

print(model.predict(x_test))

In [ ]:
def Visualize(acc,val_acc,loss, val_loss):
    fig, (ax1, ax2) = plt.subplots(nrows = 1,
                                   ncols = 2,
                                   figsize = (15,6),
                                   sharex =True)

    plot1 = ax1.plot(range(0, len(acc)),
                     acc,
                     label = 'accuracy')

    plot2 = ax1.plot(range(0, len(val_acc)),
                     val_acc,
                     label = 'val_accuracy')

    ax1.set(title = 'Accuracy And Val Accuracy progress',
            xlabel = 'epoch',
            ylabel = 'accuracy/ validation accuracy')

    ax1.legend()

    plot3 = ax2.plot(range(0, len(loss)),
                     loss,
                     label = 'loss')
    
    plot4 = ax2.plot(range(0, len(val_loss)),
                     val_loss,
                     label = 'val_loss')
    
    ax2.set(title = 'Loss And Val loss progress',
            xlabel = 'epoch',
            ylabel = 'loss/ validation loss')

    ax2.legend()

    fig.suptitle('Result Of Model', fontsize = 20, fontweight = 'bold')
    fig.savefig('Accuracy_Loss_figure.png')
    plt.tight_layout()
    plt.show()



In [ ]:
visualize_result = Visualize(hist.history['accuracy'],hist.history['val_accuracy'], hist.history['loss'], hist.history['val_loss'])

In [ ]:
y_pred=model.predict(x_test,batch_size=16,verbose=True)

In [ ]:
y_pred.shape

In [ ]:
plt.rcParams["figure.figsize"] = (12, 10)

# Définition des filtres gaussiens et de Butterworth qui vont être appliqués au dataset

In [ ]:
def filtpb_gauss(A, fc):
    M, N= A.shape
    # Image dans le domaine fréquentiel
    AA = np.fft.fftshift(np.fft.fft2(A)) 
    M0 = np.ceil((M+1) / 2)
    N0 = np.ceil((N+1)/2)
    U, V = np.mgrid[1:M+1, 1:N+1]
    D2 = (U - M0)**2 + (V - N0)**2
    # Réponse fréquentielle du filtre gaussien
    HH = np.exp(-D2 / (2 * fc**2))
    # Application du filtre et retour au domaine spatial
    BB = np.fft.ifftshift(AA*HH) 
    B = np.fft.ifft2(BB)
    B = np.real(B)
    return B

def filtpb_butter(A, fc, ordre): 
    M,N=A.shape
    # Image dans le domaine fréquentiel
    AA = np.fft.fftshift(np.fft.fft2(A)) 
    M0 = np.ceil((M+1) / 2)
    N0 = np.ceil((N+1) / 2)
    U, V = np.mgrid[1:M+1, 1:N+1]
    D2 = (U - M0)**2 + (V - N0)**2
# Réponse fréquentielle du filtre Butterworth
    HH = 1 / (1 + (D2 / fc**2)**ordre)
# Application du filtre et retour au domaine spatial
    BB = np.fft.ifftshift(AA * HH) 
    B = np.fft.ifft2(BB)
    B = np.real(B)
    return B
 

#%matplotlib inline plt.rcParams["figure.figsize"] = (12, 10)

dpiGlobal = 100
def sliders_gauss(image,a=1, fc=40):
    pb_gauss = filtpb_gauss(image, fc)
    img_gauss = image + a*(image-pb_gauss)
    #plt.figure(dpi=dpiGlobal)
    return img_gauss
#slider1 = interactive(sliders_gauss, a=(1, 3, 0.5), fc=(1, 50, 5)) 
#display(slider1)
def sliders_butter(image,a=3, fc=46, ordre=2):
    pb_butter = filtpb_butter(image, fc, ordre)
    img_butter = image + a*(image-pb_butter)
    #plt.figure(dpi=dpiGlobal)
    return img_butter


In [ ]:
img_path2 = []
last_label2 = []
for i in range(len(img_name)):
    
    img2 = cv2.imread(img_name[i], 0)
    img2 = cv2.resize(img, (224,224))
    rows, cols= img2.shape
    img2=sliders_butter(img2)
    for angle in range(360):
            M2 = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)    #Rotate 0 degree
            img_rotated2 = cv2.warpAffine(img2, M2, (224, 224))
            img_path2.append(img_rotated2)
            if label[i] == 1:
                last_label2.append(1)
            else:
                last_label2.append(0)

# Création d'un nouveau modèle utilisant les images filtrées

In [ ]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(img_path2, last_label2, test_size = 0.2, random_state = 42)

In [ ]:
x_train2 = np.array(x_train2)
x_test2 = np.array(x_test2)
(a,b,c)=x_train2.shape # (35136, 224, 224)
x_train2 = np.reshape(x_train2, (a, b, c, 1)) # 1 for gray scale
(a, b, c)=x_test2.shape
x_test2 = np.reshape(x_test2, (a, b, c, 1))

In [ ]:
model2 = model()
model2.summary()
saveModel(model3,save3)

In [ ]:
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
datagen.fit(x_train)
hist2 = model2.fit(datagen.flow(x_train2, y_train2, batch_size=32,
                 subset='training'),
                 steps_per_epoch = len(x_train2)/32,
                 validation_split=0.3,
                 epochs=5,
                 batch_size=8,
                 callbacks=[early_stop, model_check_point],validation_data=x_test2,y_test2)

In [ ]:
loss_value2 , accuracy2 = model2.evaluate(x_test, y_test)

print('Test_loss_value = ' +str(loss_value2))
print('test_accuracy = ' + str(accuracy2))

print(model.predict(x_test2))

In [ ]:
visualize_result2 = Visualize(hist2.history['accuracy'],hist2.history['val_accuracy'], hist2.history['loss'], hist2.history['val_loss'])

In [ ]:
img_path3 = []
last_label3 = []
for i in range(len(img_name)):
    
    img3 = cv2.imread(img_name[i], 0)
    img3 = cv2.resize(img3, (224,224))
    rows, cols= img3.shape
    img2=sliders_gauss(img3)
    for angle in range(360):
            M3 = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)    #Rotate 0 degree
            img_rotated3 = cv2.warpAffine(img3, M3, (224, 224))
            img_path3.append(img_rotated3)
            if label[i] == 1:
                last_label3.append(1)
            else:
                last_label3.append(0)

In [ ]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(img_path3, last_label3, test_size = 0.2, random_state = 42)

In [ ]:
x_train3 = np.array(x_train3)
x_test3 = np.array(x_test3)
(a,b,c)=x_train3.shape # (35136, 224, 224)
x_train3 = np.reshape(x_train3, (a, b, c, 1)) # 1 for gray scale
(a, b, c)=x_test3.shape
x_test3 = np.reshape(x_test3, (a, b, c, 1))

In [ ]:
model3 = model()
model3.summary()
saveModel(model3,save3)

In [ ]:
model3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
hist3 = model3.fit(datagen.flow(x_train2, y_train2, batch_size=32,
                 subset='training'),
                 steps_per_epoch = len(x_train2)/32,
                 validation_split=0.3,
                 epochs=5,
                 batch_size=8,
                 callbacks=[early_stop, model_check_point],validation_data=x_test3,y_test3)

In [ ]:
loss_value3 , accuracy3 = model3.evaluate(x_test3, y_test3)

print('Test_loss_value = ' +str(loss_value3))
print('test_accuracy = ' + str(accuracy3))

print(model.predict(x_test3))

In [ ]:
visualize_result3 = Visualize(hist2.history['accuracy'],hist2.history['val_accuracy'], hist2.history['loss'], hist2.history['val_loss'])